# Location suggestions generator for restaurant opening

## Web scraping on just-eat.com

In order create my model, I need Financial data over area of London, Demographic, Data, per area, Data about restaurants in London.

On this jupyter notebook, I wrote the code that enabled me to extract data from all take away restaurant available in London on just-eat.com to then, after connecting to my local PostgreSQL database, push the data into a table called restaurant_2.

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sqlalchemy.engine import create_engine

In [2]:
#connecting to the data base
engine=create_engine('postgresql://localhost:5432/capstone')
conn=engine.connect()

In [3]:
# this code is to get minimum you can spend at a restaurant 
def get_minimum_code(all_restaurant,i):
    try:
        code=all_restaurant[i].find_all("p",class_="c-restaurant__minimum")[0].string[-2:].encode(encoding='UTF-8',errors='strict')
        return code
    except Exception:
        return np.nan

In [4]:
# this code is to extract data about each restaurant from a district
def page_df(all_restaurant):

    name=[]
    url=[]
    ranking=[]
    num_vote=[]
    cuisine=[]
    address=[]
    min_meal=[]
    
    #online
    for i in range(len(all_restaurant)):

        #Get restaurant name
        name_resto = all_restaurant[i].find_all("h2")[0].string.encode(encoding='UTF-8',errors='strict')

        # GET Restaurant URL
        url_beg=str(all_restaurant[i].find_all("a")).find("href")
        url_end=str(all_restaurant[i].find_all("a")).find("menu")
        url_resto="https://www.just-eat.co.uk"+str(all_restaurant[i].find_all("a"))[url_beg+len("href='"):url_end-1]

        #get minimum spending
        minimum_spend_resto = get_minimum_code(all_restaurant,i)

        #get ranking:
        position_rank=str(all_restaurant[i].find_all("p",class_="c-restaurant__rating")[0]).find("stars")
        ranking_resto=str(all_restaurant[i].find_all("p",class_="c-restaurant__rating")[0])\
        [position_rank-2:position_rank+1].strip("s").strip(" ")

        num_vote_resto=all_restaurant[i].find_all("p",class_="c-restaurant__rating")[0]\
        .text.encode(encoding='UTF-8',errors='strict').strip("\n").strip("(").strip(")")

        #get cuisine:
        cuisine_resto=all_restaurant[i].find_all("p",class_="c-restaurant__cuisine")[0].string.encode(encoding='UTF-8',errors='strict')

        #get address:
        address_resto=all_restaurant[i].find_all("p",class_="c-restaurant__address")[0].string.encode(encoding='UTF-8',errors='strict')

        name.append(name_resto)
        url.append(url_resto)
        ranking.append(ranking_resto)
        num_vote.append(num_vote_resto)
        cuisine.append(cuisine_resto)
        address.append(address_resto)
        min_meal.append(minimum_spend_resto)

    df_inter=pd.DataFrame({"Name":name,"Url":url,"Ranking":ranking,"Number of votes":num_vote,\
                                  "Cuisine":cuisine, "Address":address, "Minimum spend":min_meal})
    

    return df_inter.to_sql("just_eat_4",if_exists="append",con=conn)



In [5]:
# this class is for the efficiency of the scrapig
from urllib import FancyURLopener
class MyOpener(FancyURLopener):
    version="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36\
               (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36"

In [6]:
# this function is to get the whole html information of restaurants in a district
def define_resto(url):
    page=urllib2.urlopen(url).read()
    soup=BeautifulSoup(page, "lxml")
    # get all restaurant information in a list
    #myopener = MyOpener()
    #page = myopener.open(url)
    #soup = BeautifulSoup(page)
    online = soup.find_all("div",class_="o-tile c-restaurant")
    offline = soup.find_all("div",class_="o-tile c-restaurant c-restaurant--offline")
    return online, offline

In [7]:
# L_postcode contains all disctrict postcode which is usefull to loop through just-et pages
code=pd.read_csv("L_postcode.csv")


/Users/dorian/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
district_code=code["district_code"].unique().tolist()

In [ ]:
# this code enable to retrive inforamation for all london restuarants
for code in district_code:
    url_code="https://www.just-eat.co.uk/area/"+code+"?so=distance"
    online, offline=define_resto(url_code)
    page_df(online)
    page_df(offline)
    print code